In [1]:
#@title Ejecutar primero (reiniciará el entorno)
from IPython.utils import io
with io.capture_output() as captured:
    !pip install scikit-learn==0.24.2 --upgrade
exit()

In [11]:
#@title Predicción
import pickle
import sklearn
import ipywidgets as wd
import numpy as np
from IPython.display import display, clear_output

def obtener_clasificadores():
    ruta_archivo = open(r'pickles/trabajo_v2.pkl','rb')
    clasificadores = pickle.load(ruta_archivo)
    return clasificadores

def convertir_a_variante(cod_variante, data):
    fecha = data[0][0]
    if cod_variante == 1:
        return [[fecha.isoweekday()]]
    elif cod_variante == 2:
        return [[1 if fecha.isoweekday() == i else 0 for i in range(1,8)]]
    elif cod_variante == 3:
        return [[np.sin(2 * np.pi * fecha.isoweekday() / 7), np.cos(2 * np.pi * fecha.isoweekday() / 7)]]
    elif cod_variante == 4:
        return [[fecha.month]]
    elif cod_variante == 5:
        return [[1 if fecha.month == i else 0 for i in range(1,13)]]
    elif cod_variante == 6:
        return [[np.sin(2 * np.pi * fecha.month / 12), np.cos(2 * np.pi * fecha.month / 12)]]
    elif cod_variante == 7:
        return [[fecha.isoweekday(), fecha.month]]
    elif cod_variante == 8:
        return [[fecha.isoweekday(), *[1 if fecha.isoweekday() == i else 0 for i in range(1,8)]]]
    elif cod_variante == 9:
        return [[fecha.isoweekday(), np.sin(2 * np.pi * fecha.month / 12), np.cos(2 * np.pi * fecha.month / 12)]]
    elif cod_variante == 10:
        return [[*[1 if fecha.isoweekday() == i else 0 for i in range(1,8)], fecha.month]]
    elif cod_variante == 11:
        return [[*[1 if fecha.isoweekday() == i else 0 for i in range(1,8)], *[1 if fecha.isoweekday() == i else 0 for i in range(1,8)]]]
    elif cod_variante == 12:
        return [[*[1 if fecha.isoweekday() == i else 0 for i in range(1,8)], np.sin(2 * np.pi * fecha.month / 12), np.cos(2 * np.pi * fecha.month / 12)]]
    elif cod_variante == 13:
        return [[np.sin(2 * np.pi * fecha.isoweekday() / 7), np.cos(2 * np.pi * fecha.isoweekday() / 7), fecha.month]]
    elif cod_variante == 14:
        return [[np.sin(2 * np.pi * fecha.isoweekday() / 7), np.cos(2 * np.pi * fecha.isoweekday() / 7), *[1 if fecha.isoweekday() == i else 0 for i in range(1,8)]]]
    elif cod_variante == 15:
        return [[np.sin(2 * np.pi * fecha.isoweekday() / 7), np.cos(2 * np.pi * fecha.isoweekday() / 7), np.sin(2 * np.pi * fecha.month / 12), np.cos(2 * np.pi * fecha.month / 12)]]
    elif cod_variante == 16:
        return [[fecha.isoweekday(), *[1 if fecha.isoweekday() == i else 0 for i in range(1,8)], np.sin(2 * np.pi * fecha.isoweekday() / 7), np.cos(2 * np.pi * fecha.isoweekday() / 7), fecha.month, *[1 if fecha.month == i else 0 for i in range(1,13)], np.sin(2 * np.pi * fecha.month / 12), np.cos(2 * np.pi * fecha.month / 12)]]

def mostrar_widgets(clasificadores):
    html_titulo = wd.HTML('<h3>Sugerencia de producto a vender</h3>')

    dpw_clientes = wd.Dropdown(options = clasificadores.keys(), description = 'Cliente')
    dpk_fecha_venta = wd.DatePicker(description = 'Fecha')

    btn_predecir_producto = wd.Button(description = 'Sugerir producto(s)', button_style = 'success', icon = 'check')
    def on_btn_predecir_producto_clicked(b):
        cliente = dpw_clientes.value
        clasificadores_cliente = clasificadores[cliente]
        n_clases = clasificadores_cliente['n_clases']
        clases = clasificadores_cliente['solucion']
        lista_predicciones = list()
        if n_clases == 2:
            clases_prod = clases['clases_prod']
            cod_variante = clases['cod_variante']
            clasificador = clases['clasificador']
            data = [[dpk_fecha_venta.value]]
            X = convertir_a_variante(cod_variante, data)
            prediccion = clasificador.predict(X)[0]
            producto = clases_prod[prediccion]
            lista_predicciones.append(producto)
        else:
            for producto, solucion in clases.items():
                #cod_variante, clasificador = clasificadores[dpw_clientes.value]
                cod_variante = solucion['cod_variante']
                clasificador = solucion['clasificador']

                data = [[dpk_fecha_venta.value]]
                X = convertir_a_variante(cod_variante, data)
                print(X)
                prediccion = clasificador.predict(X)[0]
                print(clasificador.predict(X))
                if prediccion == 1:
                    lista_predicciones.append(producto)

        with out_prediccion:
            clear_output()
            if lista_predicciones:
                sugerencia = ', '.join(lista_predicciones)
            else:
                sugerencia = 'No encontramos sugerencias.'
            print('Productos sugeridos: {}'.format(sugerencia))

    btn_predecir_producto.on_click(on_btn_predecir_producto_clicked)

    out_prediccion = wd.Output(layout = wd.Layout(margin = '5px 0 5px 0'))

    display(wd.VBox([html_titulo, dpw_clientes, dpk_fecha_venta, btn_predecir_producto, out_prediccion]))

def main():
    clasificadores = obtener_clasificadores()
    mostrar_widgets(clasificadores)

main()